## **5. Federated Learning**

### **Introducing PySyft**

In [0]:
!pip install syft

     |████████████████████████████████| 215kB 2.7MB/s 
     |████████████████████████████████| 71kB 27.1MB/s 
     |████████████████████████████████| 1.4MB 44.0MB/s 
     |████████████████████████████████| 389kB 40.6MB/s 
     |████████████████████████████████| 204kB 40.3MB/s 
     |████████████████████████████████| 256kB 24.5MB/s 
     |████████████████████████████████| 450kB 39.7MB/s 
     |████████████████████████████████| 276kB 38.4MB/s 
     |████████████████████████████████| 51kB 20.6MB/s 
     |████████████████████████████████| 122kB 38.1MB/s 
  Stored in directory: /root/.cache/pip/wheels/ad/9a/f4/3105b5209674ac77fcca7fede95184c62a95df0196888e0e76
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built zstd pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Found existing installation: msgpack 0.5.6
    Uninstalling msgpack-0.5.6:
     

In [0]:
import torch as th

In [0]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [0]:
y = x + x
y

tensor([ 2,  4,  6,  8, 10])

In [0]:
import syft as sy

W0626 15:27:52.073005 139693122336640 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0-rc1.so'
W0626 15:27:52.096501 139693122336640 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
hook = sy.TorchHook(th)

In [0]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

### **PySyft (Remote PyTorch)**

In Federated Learning we need to work in a number of parallel machines. Because of that, now will work with pointers instead of tensors.

In [0]:
bob = sy.VirtualWorker(hook, id="bob")

In [0]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5])

In [0]:
x = x.send(bob)
bob._objects

{44350842930: tensor([1, 2, 3, 4, 5])}

In [0]:
x.location

<VirtualWorker id:bob #objects:1>

In [0]:
x.owner

<VirtualWorker id:me #objects:0>

In [0]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [0]:
bob._objects

{}

### **Playing with remote tensors**

In [0]:
alice = sy.VirtualWorker(hook, id="alice")

In [0]:
x = th.tensor([1,2,3,4,5])

In [0]:
x_ptr = x.send(bob,alice)

In [0]:
x_ptr.get()

[tensor([1, 2, 3, 4, 5]), tensor([1, 2, 3, 4, 5])]

### **Remote arithmetic**

In [0]:
x = th.tensor([1,2,3]).send(bob)
y = th.tensor([1,1,1]).send(bob)

In [0]:
x
y
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:94425236573 -> bob:56849816558]

### **Learn a simple linear model**

In [0]:
input = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad = True).send(bob)
target = th.tensor([[1.],[1],[0],[0]], requires_grad=True).send(bob)                          

In [0]:
weights = th.tensor([[0.],[0.]], requires_grad=True).send(bob)

In [0]:
for i in range(10):
  
  pred = input.mm(weights)
  
  loss = ((pred-target)**2).sum()
  
  loss.backward()
  
  weights.data.sub_(weights.grad*0.1)
  weights.grad *= 0
  
  print(loss.get().data)

tensor(2.)
tensor(0.5600)
tensor(0.2432)
tensor(0.1372)
tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)


### **Toy Federated Learning**

In [0]:
from torch import nn, optim
import torch as th

In [0]:
# A toy dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad = True)
target = th.tensor([[1.],[1],[0],[0]], requires_grad=True) 

In [0]:
#A toy model
model = nn.Linear(2,1)

In [0]:
opt = optim.SGD(params=model.parameters(), lr = 0.1)

In [0]:
def train(iterations=20):
  
  for i in range(iterations):
    opt.zero_grad()

    pred = model(data)
    loss = ((pred-target)**2).sum()

    loss.backward()

    opt.step()

    print(loss.data)
  

train()

tensor(7.1315)
tensor(1.5408)
tensor(0.7652)
tensor(0.4783)
tensor(0.3092)
tensor(0.2011)
tensor(0.1313)
tensor(0.0860)
tensor(0.0565)
tensor(0.0373)
tensor(0.0248)
tensor(0.0165)
tensor(0.0111)
tensor(0.0075)
tensor(0.0051)
tensor(0.0035)
tensor(0.0024)
tensor(0.0017)
tensor(0.0012)
tensor(0.0008)


Now we'll make this model federated by sending the data to bob and alice and training our model there.

In [0]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

data_alice = data[0:2].send(alice)
target_alice = target[0:2].send(alice)

In [0]:
datasets = [(data_bob,target_bob), (data_alice, target_alice)]

In [0]:
def train(iterations=20):
  
  model = nn.Linear(2,1)
  opt = optim.SGD(params=model.parameters(), lr = 0.1)
  
  for i in range(iterations):
    
    for _data, _target in datasets:
      
      #sending the model
      model = model.send(_data.location)
      
      #regular training
      opt.zero_grad()

      pred = model(_data)
      loss = ((pred-_target)**2).sum()

      loss.backward()

      opt.step()
      
      #returning the model
      model = model.get()

      print(loss.get())


train()

tensor(1.1048, requires_grad=True)
tensor(0.1448, requires_grad=True)
tensor(0.1146, requires_grad=True)
tensor(0.0953, requires_grad=True)
tensor(0.0793, requires_grad=True)
tensor(0.0660, requires_grad=True)
tensor(0.0550, requires_grad=True)
tensor(0.0457, requires_grad=True)
tensor(0.0381, requires_grad=True)
tensor(0.0317, requires_grad=True)
tensor(0.0264, requires_grad=True)
tensor(0.0219, requires_grad=True)
tensor(0.0183, requires_grad=True)
tensor(0.0152, requires_grad=True)
tensor(0.0127, requires_grad=True)
tensor(0.0105, requires_grad=True)
tensor(0.0088, requires_grad=True)
tensor(0.0073, requires_grad=True)
tensor(0.0061, requires_grad=True)
tensor(0.0051, requires_grad=True)
tensor(0.0042, requires_grad=True)
tensor(0.0035, requires_grad=True)
tensor(0.0029, requires_grad=True)
tensor(0.0024, requires_grad=True)
tensor(0.0020, requires_grad=True)
tensor(0.0017, requires_grad=True)
tensor(0.0014, requires_grad=True)
tensor(0.0012, requires_grad=True)
tensor(0.0010, requi

### **Advanced Remote Execution Tools**

In [0]:
bob.clear_objects()

<VirtualWorker id:bob #objects:0>

In [0]:
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,1,1]).send(bob)

In [0]:
x = x.send(alice)

In [0]:
bob._objects

{17852148297: tensor([1, 1, 1])}

In [0]:
alice._objects

{59233375406: (Wrapper)>[PointerTensor | alice:59233375406 -> bob:17852148297]}

In [0]:
y = x + x

In [0]:
y

(Wrapper)>[PointerTensor | me:94405961767 -> alice:18102857900]

In [0]:
bob._objects

{17852148297: tensor([1, 1, 1]), 62269451727: tensor([2, 2, 2])}

This way, we are creating a chain. We have data in bob's, pointers to that data in alice's and we have pointers to alice's ones.

We can protect the data by sending the pointers to two different middle women.

In [0]:
mary = sy.VirtualWorker(hook, id="mary")

In [0]:
x = th.tensor([2,2,2]).send(bob).send(alice)
y = th.tensor([2,1,2]).send(bob).send(mary)

In [0]:
z = x+y

TensorsNotCollocatedException: ignored

That leads to an errod because although both tensors are stored in bob's, alice and mary disagree (the pointers to the data aren't stored in the same machine).

### **PointerChain Operations**

This way we can move data from bob directly to alice.

In [0]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([5,5,5]).send(bob).send(alice)

In [0]:
bob._objects

{78413604435: tensor([5, 5, 5])}

In [0]:
alice._objects

{71696799926: (Wrapper)>[PointerTensor | alice:71696799926 -> bob:78413604435]}

In [0]:
x.remote_get()

(Wrapper)>[PointerTensor | me:76483131990 -> alice:71696799926]

In [0]:
bob._objects

{}

In [0]:
alice._objects

{71696799926: tensor([5, 5, 5])}

In [0]:
# we can also use
x.move(bob)

(Wrapper)>[PointerTensor | me:76483131990 -> bob:76483131990]